## Question 2 : Optimal Market Making

In [62]:
import os
os.chdir('/Users/Alex/Desktop/Documents_4A/Winter_quarter_1/MS&E_346/RL_book/')

In [63]:
from rl.distribution import Bernoulli
import numpy as np

In [68]:
## Parameters S0 = 100, T = 1, ∆t = 0.005, γ = 0.1, σ = 2, I0 = 0, k = 1.5, c = 140 + I assume W0=0
S0, T, delta_t, gamma, sigma,I0, k, c, W0  = (100, 1, 0.005, 0.1, 2, 0, 1.5, 140, 0)

In [71]:
#delta_a = P_a - St
#delta_b = St - P_b

def OptimalAction(St,Wt,It,t):
    opti_spread = gamma * (sigma**2)*(T-t) + 2*np.log(1+gamma/k)/gamma
    opti_mid = St - It*gamma*(sigma**2)*(T-t)
    return opti_spread, opti_mid

def One_trace_opti(S0, T, delta_t, gamma, sigma,I0, k, c, W0):
    list_spread = []
    t = 0
    St, Wt, It = S0, W0, I0
    while t < T :
        opti_spread, opti_mid = OptimalAction(St,Wt,It,t)
        delta_a = (2*(opti_mid - St) + opti_spread)/2
        delta_b = (opti_spread - 2*(opti_mid - St))/2
        P_a = St + delta_a
        P_b = St - delta_b
        list_spread.append(opti_spread)
        #Impact on the PnL
        decr = Bernoulli(c*np.exp(-k*delta_a)*delta_t).sample()
        if decr:
            It = It-1
            Wt = Wt + P_a
        incr = Bernoulli(c*np.exp(-k*delta_b)*delta_t).sample()
        if incr:
            It = It + 1
            Wt = Wt - P_b
        #Change of OB Mid
        mid = Bernoulli(0.5).sample()
        if mid :
            St = St + sigma*delta_t
        else :
            St = St - sigma*delta_t
        t = t +delta_t

    proceed = Wt+It*St 
    return proceed, list_spread

In [84]:
## Calculate the average bid-ask spread
L_spread = []
l_proceed = []
N = 2000
for i in range(N):
    pro, l = One_trace_opti(S0, T, delta_t, gamma, sigma,I0, k, c, W0)
    L_spread = L_spread+l
    l_proceed.append(pro)
avg_spread = np.mean(L)
print("Average spread over the simulations : ",avg_spread)
print("Average gain with the optimal strategy : ", np.mean(l_proceed))

Average spread over the simulations :  1.491770422751422
Average gain with the optimal strategy :  65.01167698550397


In [85]:
def One_trace_naiv(S0, T, delta_t, gamma, sigma,I0, k, c, W0,avg_spread):
    t = 0
    St, Wt, It = S0, W0, I0
    while t < T :
        delta = avg_spread/2
        P_a = St + delta
        P_b = St - delta
        #Impact on the PnL
        decr = Bernoulli(c*np.exp(-k*delta)*delta_t).sample()
        if decr:
            It = It-1
            Wt = Wt + P_a
        incr = Bernoulli(c*np.exp(-k*delta)*delta_t).sample()
        if incr:
            It = It + 1
            Wt = Wt - P_b
        #Change of OB Mid
        mid = Bernoulli(0.5).sample()
        if mid :
            St = St + sigma*delta_t
        else :
            St = St - sigma*delta_t
        t = t +delta_t
    proceed = Wt + It*St
    #print("Final Proceed : ",Wt+It*St)
    return proceed

In [86]:
for i in range(N):
    pro = One_trace_naiv(S0, T, delta_t, gamma, sigma,I0, k, c, W0, avg_spread)
    l_proceed.append(pro)
print("Average gain with the naive strategy : ", np.mean(l_proceed))

Average gain with the naive strategy :  66.58290399258722
